In [8]:
import pandas as pd
from pandas import DataFrame
import os
import json

# info
Has a key that indicates whether a game was aborted or not

two staged game:
1. The questioner asks for specific information
2. the answerer is asked if the questioner knows specific key information already or not.

The main focus is on the probes (2) not the conversational stage (1).

For knowledge distillation, both sides can be considered and included in a dataset.

### Turns:
in each turn object, there is a full round of probing.
A turn begins with the questioner asking a questions to the answerer.
Afterwards there is a full round of probing all probing questions.

In [9]:
top_level_directories: list = ['v0.9', 'v1.0', 'v1.5', 'v1.5_quantized', 'v1.6', 'v1.6_backends', 'v1.6_quantized']

game: str = 'privateshared'

# Change the variants according to the game above
game_variants: list = ['3_things-places',  '2_restaurant',  '0_travel-booking', '1_job-interview', '4_letter-number']

In [10]:
os.listdir('./v0.9/claude-v1.3-t0.0--claude-v1.3-t0.0/privateshared')

['3_things-places',
 '2_restaurant',
 '0_travel-booking',
 '1_job-interview',
 '4_letter-number']

In [11]:
df_dict: dict = {
    'benchmark_version': [],
    'model_id': [],
    'difficulty': [],
    'number_of_turns': [],
    'end_of_game_type': [],
    'episode_number': [],
    'accuracy': [],
}

In [26]:
for top_level_directory in top_level_directories:
    # loop over all directories with all variants of the benchmark
    all_available_models: list = os.listdir(f'./{top_level_directory}')
    # filter all files that are not a directory
    all_available_models = [model_name for model_name in all_available_models if os.path.isdir(f'./{top_level_directory}/{model_name}')]
    for model_name in all_available_models:
        # for each model pick all episodes from the taboo game folder
        for game_variant in game_variants:
            path_to_episodes: str = f'./{top_level_directory}/{model_name}/{game}/{game_variant}'
            if not  os.path.isdir(path_to_episodes):
                continue

            all_episodes: list =  os.listdir(path_to_episodes)
            # filter all file names that do not start with episode to ensure to not read files without episodes
            all_episodes = [episode for episode in all_episodes if episode.startswith('episode')]

            # loop over all episodes to check whether an episode ended with success or any problems
            for episode in all_episodes:
                path_interactions: str = f'./{top_level_directory}/{model_name}/{game}/{game_variant}/{episode}/interactions.json'
                path_instances: str = f'./{top_level_directory}/{model_name}/{game}/{game_variant}/{episode}/instance.json'

                if os.path.isfile(path_interactions) and os.path.isfile(path_instances):
                    # open the interaction.json file to read all interactions
                    with open(path_interactions,  encoding="utf8") as json_file:
                        interactions: dict = json.load(json_file)

                        # get the last turn of the interaction
                        last_turn: list = interactions['turns'][-1]

                        # get game ending
                        game_ending: str = interactions.Aborted

                        # For privateshared, there is no target word.
                        target_word: str = last_turn[-1]['action']['data_for_computation']['target_word']

                        # add information to df_dict
                        df_dict['benchmark_version'].append(top_level_directory)
                        df_dict['model_id'].append(model_name)
                        df_dict['difficulty'].append(game_variant)
                        df_dict['number_of_turns'].append(len(interactions['turns']))
                        df_dict['end_of_game_type'].append(game_ending)
                        df_dict['episode_number'].append(episode)
                        df_dict['target_word'].append(target_word)

AttributeError: 'dict' object has no attribute 'Aborted'

In [21]:
data = pd.read_json('v0.9/claude-v1.3-t0.0--claude-v1.3-t0.0/privateshared/0_travel-booking/episode_0/scores.json')
len(data)

/var/folders/3v/xyx3_dxn5d138hmbm5hdw_qw0000gn/T/ipykernel_93023/3924334536.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  data = pd.read_json('v0.9/claude-v1.3-t0.0--claude-v1.3-t0.0/privateshared/0_travel-booking/episode_0/scores.json')
/var/folders/3v/xyx3_dxn5d138hmbm5hdw_qw0000gn/T/ipykernel_93023/3924334536.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  data = pd.read_json('v0.9/claude-v1.3-t0.0--claude-v1.3-t0.0/privateshared/0_travel-booking/episode_0/scores.json'

19

In [25]:
print(data['turn scores'].iloc[5])

{'Request Count': 6, 'Parsed Request Count': 6, 'Violated Request Count': 0, 'Accuracy': 0.6000000000000001, 'Slot Filled?': 1}
